<a href="https://colab.research.google.com/github/muziejus/coms-w4995-applied-machine-learning-project/blob/main/notebooks/combine_sentiment_and_financial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

